In [1]:
from os import path as FileLocation
from glob import glob
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
path = r'../files'
files = glob(FileLocation.join(path,'*.csv'))
data = pd.read_csv(files[0],low_memory=False)

In [3]:
data['Value'] = data['Value'].str.replace(',','').str.replace('£','').astype(float)
data['DonationType'] = data['DonationType'].str.strip()
data['AcceptedDate'] = pd.to_datetime(data['AcceptedDate'])
data['ReportedDate'] = pd.to_datetime(data['ReportedDate'])
data['ReceivedDate'] = pd.to_datetime(data['ReceivedDate'])
data['AccountingUnitName'] = data['AccountingUnitName'].fillna(' ')
data['DonorName'] = data['DonorName'].fillna('Anonymous')
data.loc[(data['DonorName'].isin(['Anonymous'])),'DonorStatus'] = 'Unidentifiable Donor'
data.loc[(data['RegulatedDoneeType'].isna()),['RegulatedDoneeType']] = 'Unknown'
data.loc[(data['CompanyRegistrationNumber'].isna()),'CompanyRegistrationNumber'] = 'Not company'
data.loc[(data['Postcode'].isna()),'Postcode'] = 'Unknown'
data.loc[(data['DonationType'].isin(['Cash'])),'NatureOfDonation'] = 'Ready money/cash'
data.loc[(data['DonationType'].isin(['Exempt Trust'])) & (data['NatureOfDonation'].isna()),'NatureOfDonation'] = data.loc[(data['DonationType'].isin(['Exempt Trust'])),'NatureOfDonation'].mode()[0]
data.loc[(data['DonationType'].isin(['Permissible Donor Exempt Trust'])) & (data['NatureOfDonation'].isna()),'NatureOfDonation'] = data.loc[(data['DonationType'].isin(['Permissible Donor Exempt Trust'])),'NatureOfDonation'].mode()[0]
data.loc[(data['DonationType'].isin(['Impermissible Donor'])) & (data['NatureOfDonation'].isna()),'NatureOfDonation'] = data.loc[(data['DonationType'].isin(['Impermissible Donor'])),'NatureOfDonation'].mode()[0]
data.loc[(data['DonationType'].isin(['Visit'])) & (data['NatureOfDonation'].isna()),'NatureOfDonation'] = 'Overtake'
data.loc[(data['DonationType'].isin(['Unidentified Donor'])) & (data['NatureOfDonation'].isna()),'NatureOfDonation'] = 'Unknown supporter'
data.loc[(data['DonationType'].isin(['Total value of donations not reported individually'])) & (data['NatureOfDonation'].isna()),'NatureOfDonation'] = 'Not reported donations'
data.loc[(data['PurposeOfVisit'].isna()),'PurposeOfVisit'] = 'Without purpose'
data.loc[(data['DonationAction'].isna()),'DonationAction'] = 'Not classified'
data.loc[(data['IsReportedPrePoll'].isna()),'IsReportedPrePoll'] = True
data['IsReportedPrePoll'] = data['IsReportedPrePoll'].astype(bool)
data.loc[(data['AccountingUnitId'].isna()),'AccountingUnitId'] = 'Unknown'
data.loc[(data['DonorId'].isna()),'DonorId'] = 'Unknown'
data.loc[(data['CampaigningName'].isna()),'CampaigningName'] = 'Unknown'
data.loc[(data['IsIrishSource'].isna()),'IsIrishSource'] = False
data['IsIrishSource'] = data['IsIrishSource'].astype(bool)
data.loc[(data['RegisterName'].isna()),'RegisterName'] = 'DonorName'

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65278 entries, 0 to 65277
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   ECRef                          65278 non-null  object        
 1   RegulatedEntityName            65278 non-null  object        
 2   RegulatedEntityType            65278 non-null  object        
 3   Value                          65278 non-null  float64       
 4   AcceptedDate                   64265 non-null  datetime64[ns]
 5   AccountingUnitName             65278 non-null  object        
 6   DonorName                      65278 non-null  object        
 7   AccountingUnitsAsCentralParty  65278 non-null  bool          
 8   IsSponsorship                  65278 non-null  bool          
 9   DonorStatus                    65278 non-null  object        
 10  RegulatedDoneeType             65278 non-null  object        
 11  CompanyRegistra

In [5]:
data.to_csv(FileLocation.join(path,'Processed_'+files[0].split('/')[len(files[0].split('/'))-1].replace(' ','_')),index=False)